# March Machine Learning Mania 2017

In this Kaggle contest, one is required to predict the outcome of NCAA Men's division I's outcome, or alternatively, the result of march mania basketball game. Given several sheets of historical basketball playing record, this year's match result is to be predicted on every possible pair among the 68 selected teams. There's no restriction on the data source, however, this solution only restricts on the dataset given by Kaggle.

The idea of my solution is quite straightforward and simple. A previous trial on prediction using seed difference only already gives good results. My solution is just improvement on this starting point.

## 1. Statistical feature generation

Following the idea that this year's tournament result be predicted on identical season's regular season performance, each team will be generated with different features in different years.

I in this solution generated 25 statistical features besides seed difference: 

1. Num of wins in regular season
2. Num of significant wins in regular season
3. Num of significant loses in regular season
4. Seed in tournament
5. Score average
6. TO average
7. AST average
8. OR average
9. DR average
10. STL average
10. BLK average
11. PF average
12. EfgO
13. EfgD
14. TopO
15. TopD
16. ORB
17. DRB
18. OFT
19. DFT
20. AstO
21. AstD
22. FGrate
23. FG3rate
24. FTrate
25. AstTo

In [1]:
import pandas as pd
import numpy as np


regular = pd.read_csv("./Data/RegularSeasonDetailedResults.csv")
regular_rev = regular.ix[:, [0,1,4,5,2,3,7,21,22,23,24,25,26,27,28,29,30,31,32,33,8,9,10,11,12,13,14,15,16,17,18,19,20]]
new_regular = pd.DataFrame(np.vstack([regular_rev.values, 
                                   regular.ix[:, [0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].values], 
                                 ), columns = ["Season","Daynum","Player","PlayerScore","Opponent","OpponentScore","Numot","Wfgm","Wfga","Wfgm3","Wfga3","Wftm","Wfta","Wor","Wdr","Wast","Wto","Wstl","Wblk","Wpf","Lfgm","Lfga","Lfgm3","Lfga3","Lftm","Lfta","Lor","Ldr","Last","Lto","Lstl","Lblk","Lpf"])
loc = []
revloc = []
for itm in regular['Wloc'].values:
    if itm=='N':
        loc.append(0)
        revloc.append(0)
    elif itm=='H':
        loc.append(1)
        revloc.append(-1)
    else:
        loc.append(-1)
        revloc.append(1)
new_regular['PlayerLoc']=np.hstack([revloc, loc])
new_regular['PlayerResult']=np.hstack([-1*np.ones(regular.shape[0], dtype=np.int), np.ones(regular.shape[0], dtype=np.int)])

In [2]:
# generate significant wins, significant lose, and location weight
marginal_thres = 3
sigwin = ((new_regular.loc[:, 'PlayerScore']-new_regular.loc[:, 'OpponentScore'])>marginal_thres).astype(np.int)
siglose = ((new_regular.loc[:, 'PlayerScore']-new_regular.loc[:, 'OpponentScore'])<-marginal_thres).astype(np.int)
locweight = (-new_regular.loc[:, 'PlayerLoc'])*0.5+1
new_regular['SigWin'] = sigwin
new_regular['SigLose'] = siglose
new_regular['LocWeight'] = locweight

In [3]:
# generate features 1,2,3
feature = new_regular.loc[:, ["Season", "Player", "PlayerResult", "SigWin", "SigLose"]].groupby(['Season', 'Player']).sum()
# generate features 5-12
feature = pd.merge(feature,
                   new_regular.loc[:, ["Season", "Player", "PlayerScore", "Wor","Wdr","Wast","Wto","Wstl","Wblk","Wpf"]].groupby(['Season', 'Player']).mean(),
                   left_index=True, right_index=True)

In [4]:
# generate features 13-26
t = pd.DataFrame({
        'EfgO': (new_regular.Wfgm+new_regular.Wfgm3/2)/new_regular.Wfga,
        'EfgD': (new_regular.Lfgm+new_regular.Lfgm3/2)/new_regular.Lfga,
        'TopO': new_regular.Wto/(new_regular.Wfga+0.44*new_regular.Wfta+new_regular.Wto),
        'TopD': new_regular.Lto/(new_regular.Lfga+0.44*new_regular.Lfta+new_regular.Lto),
        'ORB': new_regular.Wor/(new_regular.Wor+new_regular.Ldr),
        'DRB': new_regular.Wdr/(new_regular.Wdr+new_regular.Lor),
        'OFT': new_regular.Wftm/new_regular.Wfga,
        'DFT': new_regular.Lftm/new_regular.Lfga,
        'AstO': new_regular.Wast/(new_regular.Wfga+0.44*new_regular.Wfta+new_regular.Wast),
        'AstD': new_regular.Last/(new_regular.Lfga+0.44*new_regular.Lfta+new_regular.Last),
        'FGrate': new_regular.Wfgm/new_regular.Wfga,
        'FG3rate': new_regular.Wfgm3/new_regular.Wfga3,
        'FTrate': new_regular.Wftm/new_regular.Wfta,
        'AstTo': new_regular.Wast/new_regular.Wto,
        'Season': new_regular.Season,
        'Player': new_regular.Player,
    })
feature = pd.merge(feature, t.groupby(['Season', 'Player']).mean(), left_index=True, right_index=True)

In [5]:
feature.isnull().values.any()

False

In [6]:
# generate feature 4
tourney_seeds = pd.read_csv("./Data/TourneySeeds.csv")
rank = []
teams = []
for seed, team in tourney_seeds.loc[:,['Seed','Team']].values:
    rank.append(int(seed[1:3]))
tourney_seeds["NumericalSeeds"] = rank

feature = pd.merge(feature, tourney_seeds, how='left', left_index=True, right_on=['Season', 'Team'])
feature.drop('Seed', 1, inplace=True)

In [7]:
feature.head()

,PlayerResult,SigWin,SigLose,PlayerScore,Wor,Wdr,Wast,Wto,Wstl,Wblk,...,FG3rate,FGrate,FTrate,OFT,ORB,TopD,TopO,Season,Team,NumericalSeeds
2081,-4,12,14,57.250000,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,...,0.367637,0.486149,0.642402,0.291436,0.168235,0.203236,0.193121,2003,1102,NaN
2081,-1,9,9,78.777778,9.777778,19.925926,15.222222,12.629630,7.259259,2.333333,...,0.331990,0.487294,0.735271,0.342976,0.305803,0.186897,0.159805,2003,1103,NaN
1196,6,17,9,69.285714,13.571429,23.928571,12.107143,13.285714,6.607143,3.785714,...,0.325442,0.419676,0.705168,0.263855,0.371256,0.179498,0.167275,2003,1104,10.0
2081,-12,6,13,71.769231,13.500000,23.115385,14.538462,18.653846,9.307692,2.076923,...,0.359630,0.396204,0.709598,0.255629,0.335166,0.212924,0.207792,2003,1105,NaN
2081,-2,12,10,63.607143,12.285714,23.857143,11.678571,17.035714,8.357143,3.142857,...,0.350196,0.425530,0.623158,0.199740,0.349480,0.194026,0.213721,2003,1106,NaN


## 2. Unsupervised ranking feature generation

I also use several unsupervised ranking methods provided by rankit https://github.com/wattlebird/ranking to generate ranking result from different systems.

The properties to be ranked are:
1. Score
2. Efg (fgm+fgm3/2)
3. OR
4. DR
5. Ast
6. TO
7. STL
8. BLK
9. PF

And I use the following ranking methods:
1. Massey rank
2. Colley rank
3. Score difference rank
4. Markov rank (by using rate difference)
5. Markov rank (by using simple difference)
6. OD rank
7. Keener rank

In [8]:
regular.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [9]:
regular["LocWeight"] = -0.5*np.require(loc, dtype=np.float)+1
w = np.ones(regular.shape[0])
w[regular.Daynum.values>=(132-15)] = 2.0
regular["ContestWeight"] = w
regular["Weight"] = regular.LocWeight * regular.ContestWeight

In [10]:
regular['Wefgm'] = regular.Wfgm+regular.Wfgm3/2
regular['Lefgm'] = regular.Lfgm+regular.Lfgm3/2

In [40]:
score = ['score', 'efgm', 'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']
rankingfeature = feature[['Season', 'Team']]

In [35]:
# generate rank for massey
from rankit.util import Converter
from rankit.ranker import *

performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.MasseyMatrix()
        b=cvt.MasseyVector()
        ranker=MasseyRank(itemlist=cvt.ItemList())
        x=ranker.rate(M,b)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "Massey"+sc.upper()})
    performance.append(ptable)

In [41]:
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

In [43]:
# generate rank for colley
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        C=cvt.ColleyMatrix()
        b=cvt.ColleyVector()
        ranker=ColleyRank(itemlist=cvt.ItemList())
        x=ranker.rate(C,b)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "Colley"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

In [45]:
# generate rank for score difference rank
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.SymmetricDifferenceMatrix()
        ranker=DifferenceRank(itemlist=cvt.ItemList())
        x=ranker.rate(M)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "Diff"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

/home/ike/Projects/ranking/rankit/util/converter.py:202: RuntimeWarning: invalid value encountered in divide
  S = D/C


In [46]:
# generate rank for Markov rank (rate difference)
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.RateDifferenceVoteMatrix()
        ranker=MarkovRank(itemlist=cvt.ItemList(), epsilon=0.6)
        x = ranker.rate(M)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "MarkovRD"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

/home/ike/Projects/ranking/rankit/ranker/EigRank.py:27: RuntimeWarning: invalid value encountered in divide
  S = S/np.expand_dims(s, axis=1)


In [47]:
# generate rank for Markov rank (simple difference)
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.SimpleDifferenceVoteMatrix()
        ranker=MarkovRank(itemlist=cvt.ItemList(), epsilon=0.6)
        x = ranker.rate(M)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "MarkovSD"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

In [48]:
# generate rank for OD rank
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.RateVoteMatrix()
        ranker=ODRank(itemlist=cvt.ItemList(), iteration=1000000)
        x=ranker.rate(M)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "Offdef"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

In [49]:
# generate rank for Keener rank
performance = []
for y in range(2003, 2017):
    ptable = feature[feature.Season==y][['Season', 'Team']].reset_index(drop=True)
    for sc in score:
        table = pd.DataFrame({
                'primary': regular[regular.Season==y].Wteam,
                'secondary': regular[regular.Season==y].Lteam,
                'rate1': regular[regular.Season==y].loc[:, 'W'+sc],
                'rate2': regular[regular.Season==y].loc[:, 'L'+sc],
                'weight': regular[regular.Season==y].Weight
            })
        
        cvt = Converter(table=table)
        M=cvt.RateVoteMatrix()
        C = cvt.CountMatrix()
        ranker=KeenerRank(itemlist=cvt.ItemList(), epsilon=1e-5)
        x = ranker.rate(M.T, C)
        rst=ranker.rank(x)
        ptable = pd.merge(ptable, rst, left_on=['Team'], right_on=['title']).drop(['rank', 'title'], 1).rename(columns={'rate': "Keener"+sc.upper()})
    performance.append(ptable)
rankingfeature = pd.merge(rankingfeature, pd.concat(performance, ignore_index=True), on=['Season', 'Team'])

In [50]:
rankingfeature.head()

,Season,Team,MasseySCORE,MasseyEFGM,MasseyOR,MasseyDR,MasseyAST,MasseyTO,MasseySTL,MasseyBLK,...,OffdefPF,KeenerSCORE,KeenerEFGM,KeenerOR,KeenerDR,KeenerAST,KeenerTO,KeenerSTL,KeenerBLK,KeenerPF
0,2003,1102,15.854535,-2.427761,-12.310136,-7.182806,6.217532,-9.300919,4.929508,6.298500,...,512.931213,0.003070,0.003158,0.002012,0.002729,0.003553,0.002745,0.003224,0.003185,0.003012
1,2003,1103,16.253151,-5.175964,-10.910529,-6.213183,1.314330,-9.728739,4.416701,5.547285,...,449.173828,0.003342,0.003218,0.003120,0.003225,0.003278,0.003149,0.003221,0.003141,0.003116
2,2003,1104,26.567745,0.122350,-3.984939,-0.261003,5.122419,-8.491093,6.204427,7.733090,...,475.630524,0.004490,0.004380,0.004632,0.004456,0.004526,0.003945,0.004608,0.004810,0.003843
3,2003,1105,-2.849162,-9.943722,-7.421961,-12.749739,-1.969988,-7.281213,3.484138,3.212066,...,626.148315,0.002130,0.002178,0.002265,0.002085,0.002024,0.002457,0.002535,0.001871,0.002679
4,2003,1106,3.725306,-5.328582,-8.643593,-7.257001,-0.731972,-4.862918,3.706864,6.063754,...,665.593323,0.002243,0.002383,0.002436,0.002229,0.002134,0.002529,0.002480,0.002401,0.002750


In [51]:
feature = pd.merge(feature, rankingfeature, on=['Season', 'Team'], sort=True).reset_index(drop=True)

In [53]:
feature.columns

Index([u'PlayerResult', u'SigWin', u'SigLose', u'PlayerScore', u'Wor', u'Wdr',
       u'Wast', u'Wto', u'Wstl', u'Wblk', u'Wpf', u'AstD', u'AstO', u'AstTo',
       u'DFT', u'DRB', u'EfgD', u'EfgO', u'FG3rate', u'FGrate', u'FTrate',
       u'OFT', u'ORB', u'TopD', u'TopO', u'Season', u'Team', u'NumericalSeeds',
       u'MasseySCORE', u'MasseyEFGM', u'MasseyOR', u'MasseyDR', u'MasseyAST',
       u'MasseyTO', u'MasseySTL', u'MasseyBLK', u'MasseyPF', u'ColleySCORE',
       u'ColleyEFGM', u'ColleyOR', u'ColleyDR', u'ColleyAST', u'ColleyTO',
       u'ColleySTL', u'ColleyBLK', u'ColleyPF', u'DiffSCORE', u'DiffEFGM',
       u'DiffOR', u'DiffDR', u'DiffAST', u'DiffTO', u'DiffSTL', u'DiffBLK',
       u'DiffPF', u'MarkovRDSCORE', u'MarkovRDEFGM', u'MarkovRDOR',
       u'MarkovRDDR', u'MarkovRDAST', u'MarkovRDTO', u'MarkovRDSTL',
       u'MarkovRDBLK', u'MarkovRDPF', u'MarkovSDSCORE', u'MarkovSDEFGM',
       u'MarkovSDOR', u'MarkovSDDR', u'MarkovSDAST', u'MarkovSDTO',
       u'MarkovSDSTL', u'Mark

## 3. Feature normalization

## 4. Feature Selector and training set generator

## 5. Model building

## 6. Result generation